<a href="https://colab.research.google.com/github/blue-mocha/books_recommand/blob/main/%EB%8F%84%EC%84%9C%EC%B6%94%EC%B2%9C_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


#유저정보 예시 (좋아요 리스트 - 책제목, 책소개)-------------------------------------------------#
title_1 = ['살롱 드 경성', '방구석 미술관', '이구아나']
intro_1 = ['''〈김인혜의 살롱 드 경성〉을 드디어 책으로 만나다!“이 책은 우리 근대문화사의 소중한 증언록이다.”_유홍준
식민지 암흑기와 전쟁의 비극 속에 삶은 부서졌지만 예술을 향한 그들의 집념과 열정은 멈추지 않았다
근원으로 돌아가 삶의 열망과 존재의 이유를 뜨겁게 되묻는 한국 근대기 예술가들의 슬프도록 찬란한 유산
예술이 삶이 되고 삶이 예술이 되다! 가장 헐벗고 참혹했던 순간에도 문학과 미술을 꽃피운
한국 근대 예술가들의 세계로 초대합니다!오늘날 한국 미술계를 향한 전 세계의 관심이 뜨겁다. 프리즈 등 세계적 아트페어가 서울에서 열려 문전성시를 이루고,
김환기 등 한국 화가의 작품이 100억이 넘는 가격에 낙찰되기도 한다.
이처럼 불과 100여 년 만에 한국 미술이 안팎으로 급성장하기까지, 열악한 환경에서도 예술혼을 불태웠던 선구자들이 있었다.
바로 19세기 말부터 1950년대까지 과도기에 활약했던 근대 미술가들이다. 그럼에도 우리는 근대기 한국 작가들이라고 하면 이중섭과 박수근 정도만 떠올릴 뿐, 아는 바가 많지 않다.
이에 국립현대미술관의《미술이 문학을 만났을 때》《이중섭 백년의 신화》《내가 사랑한 미술관》《윤형근》등 블록버스터 전시를 기획했던 큐레이터 김인혜가
한국 근대사를 수놓은 천재 화가들의 생애와 작품 세계를 정리한『살롱 드 경성』을 펴냈다.'''
]


#웹크롤링(교보문고, 예술분야 베스트셀러)------------------------------#
import requests
import json

#json 데이터 웹크롤링
url = '''https://product.kyobobook.co.kr/api/gw/pdt/best-seller/online?page=1&per=100
&saleCmdtDvsnCode=KOR&saleCmdtClstCode=23&saleCmdtDsplDvsnCode=KOR&period=002&dsplDvsnCode=001&dsplTrgtDvsnCode=004'''
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
page = requests.get(url,headers=headers)
json_data = json.loads(page.text)

#필요_데이터 부분수집(책제목,책소개 부분)
title_2 = []
intro_2 = []
for i in range(len(json_data['data']['bestSeller'])):
 title_2.append(json_data['data']['bestSeller'][i]['cmdtName'])
 intro_2.append(json_data['data']['bestSeller'][i]['inbukCntt'])


#중복도서 제거---------------------------------------------------------------#
dupe = list(set(title_1).intersection(title_2))
dupe_idx = []

for i in range(len(dupe)):
 dupe_idx.append(title_2.index(dupe[i]))

if len(dupe_idx) > 0 :
 title_2 = np.delete(title_2,dupe_idx)
 intro_2 = np.delete(intro_2,dupe_idx)

print("중복도서",dupe)

#책제목 + 책소개 합치기
doc1 = " ".join(title_1) + " ".join(intro_1).replace("\n", "")
doc2 = []
for i in range(len(title_2)):
   doc2.append(title_2[i] + ' ' + str(intro_2[i]).replace("\n", ""))


#텍스트 토큰나이징
doc1_tokenized = doc1.split()
doc2_tokenized = list(map(lambda x: x.split(), doc2))


#자카드 유사도-------------------------------------------------#

jaccard_similarity = [] # 유사도 결과 리스트

for i in range(len(doc2_tokenized)):

 doc_union = set(doc1_tokenized).union(set(doc2_tokenized[i])) # 합집합
 doc_intersection = set(doc1_tokenized).intersection(set(doc2_tokenized[i])) # 교집합
 jaccard_similarity.append (len(doc_intersection) / len(doc_union)) #합집합 대 교집합의 비율


#유사도 sort 후,도서제목 리스트 받아내기.
sorted_idx = np.array(jaccard_similarity).argsort()[::-1]
sorted_idx = sorted_idx[: 5]


#추천도서(결과)

result_1 = []
for i in sorted_idx:
 result_1.append(title_2[i])
print("자카드유사도 결과",sorted_idx, result_1) # >> 노드 서버에서 검색해서 제공.(좋아요 목록 업데이트 전까지 캐시유지)


##코사인 유사도----------------------------------------------------------------#

count_vectorizer = TfidfVectorizer() #2차원
feature_vect_simple = count_vectorizer.fit_transform([doc1] + doc2)
cos_res = cosine_similarity(feature_vect_simple[0] , feature_vect_simple)

sorted_index = cos_res.argsort()[::-1] #::-1 역순 // 크기별로 인덱스 붙이기
sorted_index = sorted_index[:,1: 6][0] #[[28 78 32 35 36]]

result_2 = []
for i in sorted_index:
 result_2.append(title_2[i])

print("코사인유사도 결과",sorted_index, result_2) #유사도가 높은 순으로 인덱스 반환

#------------------------------------------------------------------------------

#[결과 예시]

#자카드유사도 > 코사인 유사도 : 키워드가 중요한 상황으로, 비슷한 카테고리(미술)에 관한 책들을 추천함.

#자카드유사도 결과 [95 28 17  5 10] ['그림들', '스즈키 바이올린 교본 1', '화가가 사랑한 바다', '거의 모든 순간의 미술사', '방구석 미술관 2: 한국']
#코사인유사도 결과 [28 78 32 35 36] ['스즈키 바이올린 교본 1', '공간력 수업', '미니 캐릭터 그리는 법', '스즈메의 문단속 공식 비주얼 가이드', '오펜하이머 각본집'


#------------------------------------------------------------------------------

중복도서 ['살롱 드 경성', '방구석 미술관']
자카드유사도 결과 [95 28 17  5 10] ['그림들', '스즈키 바이올린 교본 1', '화가가 사랑한 바다', '거의 모든 순간의 미술사', '방구석 미술관 2: 한국']
코사인유사도 결과 [28 78 32 35 36] ['스즈키 바이올린 교본 1', '공간력 수업', '미니 캐릭터 그리는 법', '스즈메의 문단속 공식 비주얼 가이드', '오펜하이머 각본집']
